# TD TN20 : Roulements
## Broche de machine outil

<img src="broche_machine_outil.png" width="700"/>
Donnée sur la géométrie de la broche :
$$\lambda=\frac{a}{b}=0.275$$

#### 1) Actions mécaniques supportées par les roulements :

Calcul classique, on écrit le PFD en supposant un régime de rotation stationnaire et un arbre parfaitement équilibré statiquement et dynamiquement.

$$
\{\mathcal{T}_{\text{Ext.}\to\text{Arbre}}\}=\{0\}
$$

$$
\{\mathcal{T}_{\text{Effort}\to\text{Arbre}}\} +
\{\mathcal{T}_{\text{Rot. (A)}\to\text{Arbre}}\} +
\{\mathcal{T}_{\text{Lin. An. (B)}\to\text{Arbre}}\}
=\{0\}
$$

$$
\left\lbrace
\begin{array}{c}
\vec{F}_x+\vec{F}_y\\
\vec{AO}\wedge\left(\vec{F}_x+\vec{F}_y\right)
\end{array}
\right\rbrace_{A}
+
\left\lbrace
\begin{array}{c}
\vec{\mathcal{R}_{A}}\\
\vec{0}
\end{array}
\right\rbrace_{A}
+
\left\lbrace
\begin{array}{c}
\vec{\mathcal{R}_{B}}\\
\vec{AB}\wedge\vec{\mathcal{R}_{B}}
\end{array}
\right\rbrace_{A}
=
\left\lbrace
\begin{array}{c}
\vec{0}\\
\vec{0}
\end{array}
\right\rbrace
$$

$$
\left\lbrace
\begin{array}{crcl}
F_x&-a& &F_x\\
F_y&0&\wedge&F_y\\
0&0& &0
\end{array}
\right\rbrace_{A}
+
\left\lbrace
\begin{array}{cc}
X_{A}&0\\
Y_{A}&0\\
Z_{A}&0
\end{array}
\right\rbrace_{A}
+
\left\lbrace
\begin{array}{crcl}
0&b& &0\\
Y_{B}&0&\wedge&Y_{B}\\
Z_{B}&0& &Z_{B}
\end{array}
\right\rbrace_{A}
=
\left\lbrace
\begin{array}{rl}
0&0\\
0&0\\
0&0
\end{array}
\right\rbrace
$$

$$
\left\lbrace
\begin{array}{rl}
F_x + X_A &=0\\
F_y + Y_A + Y_B &=0\\
Z_{A} + Z_{B} &=0\\
0 &=0\\
-b.Z_B&=0\\
a.F_y + b.Y_B&=0
\end{array}
\right.
$$

On est bien dans le cas d'un système isostatique (6 équations scalaires du PFD, 5 inconnues d'efforts et un degré de mobilité).

$$
\{\mathcal{T}_{\text{Rot. (A)}\to\text{Arbre}}\}:
\left\lbrace
\begin{array}{cc}
X_{A}=-F_x&0\\
Y_{A}=-(1+\lambda)F_y&0\\
Z_{A}=0&0
\end{array}
\right\rbrace_{A}
\quad
\{\mathcal{T}_{\text{Lin. An. (B)}\to\text{Arbre}}\}:
\left\lbrace
\begin{array}{cc}
X_{B}=0&0\\
Y_{B}=-\lambda F_y&0\\
Z_{B}=0&0
\end{array}
\right\rbrace_{B}
$$

#### 2) Charges radiales équivalentes pour chacune des 3 phases de fonctionnement :

<div class="alert alert-info">
<img src="verif_duree_vie_rotule_rouleau_oblique.svg" width="600">
    </div>

In [1]:
import numpy as np
# Données roulements
# 1208ETN9
dA = 40
DA = 80
BA = 18
CA = 19900
C0A = 6950
eA = 0.22
XA = 0.65
Y1A = 2.9
Y2A = 4.5
# 6208
dB = 40
DB = 80
BB = 18
CB = 32500
C0B = 19000
# Données chargement broche machine outil :
Fx = [100,280,340]
Fy = [400,410,565]
Phase =[[800,0.20],[1200,0.50],[1800,0.30]]
L = 0.275

# Fonction de calcul de la charge radiale équivalente
def Peq(Fa,Fr,e,X,Y1,Y2):
    if ((Fa/Fr)<=e):
        P = Fr + Y1*Fa
    else:
        P = X*Fr + Y2*Fa
    return P

LPA = []
LPB = []
# Pour toutes les phases de chargement ...
for i,(fx,fy) in enumerate(zip(Fx,Fy)):
    FrA = np.abs(-(1+L)*fy)
    FaA = np.abs(-fx)
    PA = Peq(FaA,FrA,eA,XA,Y1A,Y2A)
    PB = np.abs(-L*fy)
    print('Phase %i | Peq A %8.2f N | Peq B %8.2f N'%(i,PA,PB))
    LPA.append(PA)
    LPB.append(PB)
    

Phase 0 | Peq A   800.00 N | Peq B   110.00 N
Phase 1 | Peq A  1599.79 N | Peq B   112.75 N
Phase 2 | Peq A  1998.24 N | Peq B   155.38 N


#### 3) Charges radiales équivalentes sur l'ensemble des 3 phases :

<div class="alert alert-info">
    Loi de Miner (cumul de l'endommagement) appliquée au cas des roulements :
$$
P_\text{eq}=\left(\frac{\sum\limits_{i}U_i.P_i^n}{\sum\limits_{i}U_i}\right)^\frac{1}{n}
$$
Avec :
<ul>
     <li>$P_\text{eq}$ : charge équivalente sur l'ensemble des phases de chargement.</li>
    <li>$U_i=\omega_i.t_i$ : $t_i$ durée de phase $i$ à la vitesse de rotation $\omega_i$ $\to$ en nombre de tours pour un roulement.</li> 
    <li>$P_i$ : charge radiale équivalente supportée durant la phase $i$.</li>
    <li>$n$ : type de roulement, $n=$3 pour les billes, $n=$10/3 pour les rouleaux.</li>     
</ul>
</div>

In [2]:
# Initialise les variables pour les sommes ..
Utot = 0   # Somme des Ui
UPtotA = 0 # Somme des Ui.Pi pour le rlt A.
UPtotB = 0 # Somme des Ui.Pi pour le rlt B.
# Calcul des sommes pour les 3 phases ..
for phasei,PAi,PBi in zip(Phase,LPA,LPB):
    Ui = phasei[0]*phasei[1]
    Utot = Utot + Ui
    UPtotA = UPtotA + Ui*(PAi**3)
    UPtotB = UPtotB + Ui*(PBi**3)

PeqA = (UPtotA/Utot)**(1/3)
PeqB = (UPtotB/Utot)**(1/3)
print('Charge radiale équiv.\n\tRlt. A %8.2f N\n\tRlt. B %8.2f N'%(PeqA,PeqB))

Charge radiale équiv.
	Rlt. A  1739.89 N
	Rlt. B   133.58 N


#### 4) Durée de vie de chaque roulement pour une fiabilité de 90% par roulement :

In [13]:
L10A = (CA/PeqA)**3
L10B = (CB/PeqB)**3
print('Durée de vie à 90%% en million de tours:\n\t*Rlt. A %.0f\n\t*Rlt. B %.0f'%(L10A,L10B))

Durée de vie à 90% en million de tours:
	*Rlt. A 1496
	*Rlt. B 14402691


Remarque :<br>
Avec une fiabilité de 90% par roulement la liaison a une fiabilité $f_\text{Liaison}=0.9^2=0.81$.

#### 5) Durée de vie pour une fiabilité de 95%  de la liaison

On reprend la relation du TD sur la fiabilité d'une liaison à 3 roulements, on l'applique ici au cas des roulements A et B :

$$
L_\text{Liaison} = \left[ \frac{\ln\left(f_\text{Liaison}\right)}{\ln 0.9\left({L_{10A}}^{-\frac{3}{2}}+{L_{10B}}^{-\frac{3}{2}}\right)} \right]^\frac{2}{3}
$$

In [19]:
sumL10 = L10A**(-3/2) + L10B**(-3/2)
L = ( np.log(0.95)/(np.log(0.9)*sumL10) )**(2/3)
print('Durée de vie :\n\t*En millions de tours : %.f'%L)

Durée de vie :
	*En millions de tours : 926


Pour calculer la durée de vie en heures on utilise le nombre de tours en 1 minute sur l'ensemble des 3 phases soit : $\sum\limits_{i}U_i=\sum\limits_{i}\omega_i.t_i$

In [21]:
# Utot est le nbre de tour en une minute pour les 3 phases -> somme des Ui
Lh = L*1e6/(Utot*60)
print('Durée de vie :\n\t*En heures : %.2f h'%Lh)

Durée de vie :
	*En heures : 11870.94 h
